# update6 - TBD

In [1]:
import sys

In [2]:
sys.path.append('../..')

In [3]:
from datetime import datetime, timezone
import json
from tqdm import tqdm
from pathlib import Path
import gzip
import pickle

In [4]:
import pandas as pd
import numpy as np

In [5]:
import rapidfuzz

In [6]:
from app import db
from models import Source, ISSNtoISSNL
from models.source import DELETED_SOURCE_ID
from sqlalchemy import text
from sqlalchemy.orm import Load
from sqlalchemy.exc import MultipleResultsFound

140458990035328: loading merged_into_institutions_dict
140458990035328: loading valid concept IDs
140458990035328: loading valid topic IDs
140458990035328: loading valid keyword IDs


In [7]:
from cleanup.util import make_request, paginate_openalex


In [8]:
%%time
sq = """select if.*, ife.cluster_title from issn_audit_20240321.issn_ic_datafile_202402 if
left join issn_audit_20240321.issn_ic_datafile_expanded_202402 ife
  on if."submitted_1348-0278"  = ife.issns;"""
df_issnl_file = pd.read_sql_query(sq, db.engine)

CPU times: user 578 ms, sys: 197 ms, total: 775 ms
Wall time: 2.54 s


In [9]:
df_issnl_file = df_issnl_file.rename(columns={"submitted_1348-0278": "submitted_issn"})

In [10]:
%%time
sq = """select journal_id, display_name, issn, issns, issns_text_array, type, country_code, alternate_titles, publisher_id
    from mid.journal
    where merge_into_id is null"""
df_midjournal = pd.read_sql_query(sq, db.engine)

CPU times: user 1.58 s, sys: 109 ms, total: 1.69 s
Wall time: 2.9 s


In [11]:
print(len(df_midjournal))
df_midjournal.dropna(subset='issn', inplace=True)
print(len(df_midjournal))

252999
139752


In [12]:
smap = {}
for source_id, issn_list in tqdm(df_midjournal.set_index('journal_id', verify_integrity=True)['issns_text_array'].items(), total=len(df_midjournal)):
    for issn in issn_list:
        if issn in smap:
            smap[issn].append(source_id)
        else:
            smap[issn] = [source_id]

  0%|          | 0/139752 [00:00<?, ?it/s]

100%|██████████| 139752/139752 [00:00<00:00, 504135.06it/s]


In [13]:
d = []
for issn, source_list in smap.items():
    d.append({
        'issn': issn,
        'num_sources': len(source_list),
    })
_df = pd.DataFrame(d)
_df['num_sources'].value_counts()
df_issnl_file['num_sources_resolve'] = df_issnl_file['submitted_issn'].map(_df.set_index('issn', verify_integrity=True)['num_sources'])
df_issnl_file['num_sources_resolve'].fillna(value=0, inplace=True)

In [14]:
# actual_works_count = []
# subset = {issn: source_list for issn, source_list in smap.items() if len(source_list) > 1}
# for issn, source_list in tqdm(subset.items()):
#     if len(source_list) > 1:
#         for source_id in source_list:
#             url = f'https://api.openalex.org/works?filter=locations.source.id:S{source_id}'
#             params = {'mailto': 'jportenoy@ourresearch.org',
#                       'select': 'id',
#                       'per-page': 1}
#             r = make_request(url, params=params)
#             this_c = r.json()['meta']['count']
#             actual_works_count.append({
#                 'issn': issn,
#                 'source_id': source_id,
#                 'works_count': this_c,
#             })
# df_actual_works_count = pd.DataFrame(actual_works_count)

In [15]:
df_actual_works_count = pd.read_pickle('../data/issn_audit_20240301/df_actual_works_count.pickle')

In [16]:
x = df_actual_works_count[['source_id', 'works_count']].drop_duplicates()
x = x.set_index('source_id', verify_integrity=True)['works_count']
df_midjournal['works_count'] = df_midjournal['journal_id'].map(x)

In [17]:
# %%time
# url = "https://api.openalex.org/sources"
# params = {
#     'mailto': 'jportenoy@ourresearch.org',
#     'group_by': 'issn',
#     'bypass_cache': 'true',
# }
# data = []
# for r in paginate_openalex(url, params=params):
#     data.extend(r.json()['group_by'])

In [18]:
# df_sources = pd.DataFrame(data)
# _rename = {
#     'key': 'issn',
#     'count': 'num_sources_in_openalex',
# }
# df_openalex_issn_sources_count = df_sources.rename(columns=_rename).drop(columns=['key_display_name'])

In [19]:
# df_openalex_issn_sources_count['num_sources_in_openalex'].value_counts()

In [28]:
df_issn_to_issnl = pd.read_sql_query("""select * from mid.journal_issn_to_issnl""", db.engine)

In [29]:
issn_to_issnl = df_issn_to_issnl.set_index('issn', verify_integrity=True)['issnl']

In [30]:
issnls = issn_to_issnl.unique()

In [32]:
# get dict of issnl to issn
x = df_issn_to_issnl.dropna(subset=['issnl'])
# x = x[x['category'].isin(['Register', 'Work', 'Free', 'Validation Request'])]
x = x[['issn', 'issnl']]
issnl_to_issn = {}
ignore = [
    '0000-0000',
]
x = x[~(x['issnl'].isin(ignore))]
for name, gbdf in x.groupby('issnl'):
    if name in ignore:
        continue
    issnl_to_issn[name] = gbdf['issn'].tolist()
len(issnl_to_issn)

141008

In [46]:
print(len(df_issnl_file))
df_issn_gold = df_issnl_file[['issnl', 'cluster_title']].dropna().drop_duplicates()
print(len(df_issn_gold))
df_issn_gold = df_issn_gold[df_issn_gold['cluster_title']!='None']
print(len(df_issn_gold))
print(len(df_issn_gold['issnl'].drop_duplicates()))

214606
140887
140887
140887


In [64]:
round(len(df_midjournal) / 20)

6988

In [75]:
%%time
issn_to_title = df_issn_gold.set_index('issnl', verify_integrity=True)['cluster_title']
try:
    num_updates = 0
    i = 0
    for issnl, row in tqdm(df_midjournal.set_index('issn', verify_integrity=True).iterrows(), total=len(df_midjournal)):
        updated_date = datetime.now(timezone.utc).isoformat()
        do_update = False
        title = issn_to_title.get(issnl)
        old_display_name = row['display_name']
        if not title or title == old_display_name:
            new_display_name = old_display_name
        else:
            new_display_name = title
            do_update = True
        
        issn_list = issnl_to_issn[issnl]
        if issnl not in issn_list:
            issn_list.append(issnl)
        # move issnl to the front
        issn_list.insert(0, issn_list.pop(issn_list.index(issnl)))
        old_issn_list = row['issns_text_array']
        if issn_list == old_issn_list:
            new_issn_list = old_issn_list
        else:
            new_issn_list = issn_list
            do_update = True
        
        if do_update is True:
            # make updates
            source_id = row['journal_id']
            source = db.session.query(Source).filter_by(journal_id=source_id).one()
            source.display_name = new_display_name
            source.issns = json.dumps(new_issn_list)
            source.issns_text_array = new_issn_list
            source.updated_date = updated_date
            db.session.add(source)

            # insert into log table
            sq = """INSERT INTO issn_audit_20240321.update6
                (issnl, old_issns, new_issns, old_display_name, new_display_name, updated_date, source_id)
                VALUES(:issnl, :old_issns, :new_issns, :old_display_name, :new_display_name, :updated_date, :source_id);
                """
            db.session.execute(text(sq), {
                'issnl': issnl,
                'old_issns': row['issns'],
                'new_issns': json.dumps(issn_list),
                'old_display_name': old_display_name,
                'new_display_name': new_display_name,
                'updated_date': updated_date,
                'source_id': source.id,
            }) 
            num_updates += 1
        
        for issn in issn_list:
            sq = """update issn_audit_20240321.issn_ic_datafile_202402 set resolved = true
                where issn = :issn
                """
            db.session.execute(text(sq), {
                'issn': issn,
            })
        
        i += 1
        if i % round(len(df_midjournal) / 20) == 0:
            db.session.commit()

    # final commit for anything left
    db.session.commit()
finally:
    db.session.close()


100%|██████████| 139752/139752 [2:02:49<00:00, 18.96it/s]  


CPU times: user 3min 10s, sys: 14.7 s, total: 3min 25s
Wall time: 2h 2min 51s


In [76]:
num_updates

4764

In [ ]:
# def update_source(issnl, issn_list, title, updated_date=None, session=None):
#     if session is None:
#         session = db.session
#     if updated_date is None:
#         updated_date = datetime.now(timezone.utc).isoformat()
#     source = db.session.query(Source).filter_by(journal_id=id_to_merge).one()
#     if source is None:
#         raise KeyError(f"issnl {issnl} not found in db")
#     old_issnl = source.issn
#     old_display_name = source.display_name
#     old_issns = source.issns

#     # make updates
#     if title:
#         source.display_name = title
#     source.issn = issnl
#     source.issns = json.dumps(issn_list)
#     source.issns_text_array = issn_list
#     # source.type = source_type
#     source.updated_date = updated_date
#     session.add(source)

#     # insert into log table
#     sq = """INSERT INTO issn_audit_20240321.update3
#         (issnl, old_issns, new_issns, old_issnl, old_display_name, new_display_name, updated_date, source_id)
#         VALUES(:issnl, :old_issns, :new_issns, :old_issnl, :old_display_name, :new_display_name, :updated_date, :source_id);
#         """
#     session.execute(text(sq), {
#         'issnl': issnl,
#         'old_issns': old_issns,
#         'new_issns': json.dumps(issn_list),
#         'old_issnl': old_issnl,
#         'old_display_name': old_display_name,
#         'new_display_name': title if title else old_display_name,
#         'updated_date': updated_date,
#         'source_id': source.id,
#     }) 

In [ ]:
# try:
#     # for issnl, issn_list in issnl_to_issn.items():
#     # for issnl, issn_list in tqdm(issnl_to_issn.items(), total=len(issnl_to_issn)):
#     for issnl in tqdm(multfound['issnl'].values, total=len(multfound)):
#         now = datetime.now(timezone.utc).isoformat()
#         issn_list = issnl_to_issn[issnl]
#         title = issnl_display_name[issnl]
#         try:
#             if issnl not in issn_list:
#                 issn_list.append(issnl)
#             # move issnl to the front
#             issn_list.insert(0, issn_list.pop(issn_list.index(issnl)))
#             update_source(issnl, issn_list, title, updated_date=now, session=db.session)
#             for issn in issn_list:
#                 sq = """update issn_audit_20240321.issn_ic_datafile_202402 set resolved = true
#                     where issn = :issn
#                     """
#                 db.session.execute(text(sq), {
#                     'issn': issn,
#                 })
#         except KeyError:
#             continue
#         except MultipleResultsFound:
#             continue

#     db.session.commit()
# finally:
#     db.session.close()